<a href="https://colab.research.google.com/github/YeonKang/Tensorflow-with-Colab/blob/master/Extra__tf_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import traceback
import contextlib

#Helper function for error output
@contextlib.contextmanager
def assert_raises(error_class):
  try:
    yield
  except error_class as e:
    print('Throwing the expected exception \n  {}:'.format(error_class))
    traceback.print_exc(limit=2)
  except Exception as e:
    raise e
  else:
    raise Exception('Expected {}, but no error occurred'.format(
        error_class))

**calcultate gradient descent with tf.function**

In [ ]:
@tf.function
def add(a, b):
  return a + b

add(tf.ones([2, 2]), tf.ones([2, 2]))  #  [[2., 2.], [2., 2.]]

In [ ]:
v = tf.Variable(1.0)
with tf.GradientTape() as tape:
  result = add(v, 1.0)
tape.gradient(result, v)

**use tf.function in another function**

In [ ]:
@tf.function
def dense_layer(x, w, b):
  return add(tf.matmul(x, w), b)

In [ ]:
dense_layer(tf.ones([3, 2]), tf.ones([2, 2]), tf.ones([2]))